In [42]:
import re
import requests
from bs4 import BeautifulSoup 
import pickle
import json
from tqdm import tqdm
import torch
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer, util
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

### this set up is to intergrate the langchain on already existed Elasticsearch index
#### dependencies as follows
#### langchain = 0.0.190
#### python 3.11.0


In [43]:
client = Elasticsearch(
     cloud_id="",
     basic_auth=("", ""),
    )

embedder = SentenceTransformer('sentence-transformers/msmarco-roberta-base-v2')
device = torch.device('cuda:0')

In [55]:
def text_similarity_search(question):
  user_query=question
  query_vector=embedder.encode([user_query][0]).tolist()
  response_knn = client.search(
          index="your_ES_Index_Name",
          body={   
          "size":1 ,     
            "knn": {
                  "field": "para_vector",
                  "query_vector": query_vector,
                  "k": 10,
                  "num_candidates": 100
                  
                  }
              }                
              )
  result=response_knn['hits']['hits'][0]['_source']['your_text_key']
  return result

In [82]:


topic = "search"
index_name = "book_wookieepedia_small1"

cache_dir = "./cache"

def getFlanLarge():
    model_id = 'google/flan-t5-large'
    print(f">> Prep. Get {model_id} ready to go")
    tokenizer = AutoTokenizer.from_pretrained(model_id) 
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_id, cache_dir=cache_dir) 
    
    pipe = pipeline(
        "text2text-generation",
        model=model, 
        tokenizer=tokenizer, 
        max_length=100
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

def make_the_llm():
    template_informed = """
      AI that answers questions. When I don't know the answer withsome meaningfull words. 
    I know context: {context}
    when asked: {question}
    my response using only information in the context is: """

    prompt_informed = PromptTemplate(template=template_informed, input_variables=["context", "question"])

    llm = getFlanLarge()

    return LLMChain(prompt=prompt_informed, llm=llm)

llm_chain_informed= make_the_llm()


def sumary_llm():
    prompt_template = """Write a summary of the following:

    {text}"""
    
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
    return  load_summarize_chain(getFlanLarge(), chain_type="stuff", prompt=PROMPT)
   

llm_summary=sumary_llm()

>> Prep. Get google/flan-t5-large ready to go
>> Prep. Get google/flan-t5-large ready to go


## generate QnA and Summary for the given question based on the nest extracted output from ES

In [ ]:

## how to ask a question
def ask_a_question(question):
    similar_docs = text_similarity_search(question)
    informed_context= similar_docs
    informed_response = llm_chain_informed.run(context=informed_context,question=question)
    summary=llm_summary.run([Document(page_content=similar_docs, metadata={})])
    
    return [informed_response,informed_context,summary]




print(f'Ask me any question about {topic}')

while True:
    question = input("User Question >> ")
    
    response= ask_a_question(question)[0]
    print('contaxt :',ask_a_question(question)[1])
    print('summary :',ask_a_question(question)[2])
    print(f"\tAnswer  : {response}")
    break
